In [99]:
import pandas as pd
import os
import glob

directory = '/Users/femartinez/Documents/Research/DecodingFatphobia/data'

files = glob.glob(os.path.join(directory, '* image ratings.xlsx'))

dataframes = []
missing_rows_log = []
# 
def to_tuple(value):
    '''
    Convert values to tuples
    '''
    if isinstance(value, (int, float)):
        return (value,)
    elif isinstance(value, str):
        return tuple(map(str.strip, value.split(',')))
    return (value,)

# Loop through each file
for file in files:
   
    rater = str.upper(file.split('/')[-1].split(' ')[0]) # based on file name i.e.: 'final JANE image ratings - decoding fatphobia.xlsx'
#     # Read the Excel file
    excel_file = pd.ExcelFile(file)
    
#     # Loop through each sheet in the file
    for sheet_name in excel_file.sheet_names:
#         # Read the sheet data, handling variable number of columns
        df = pd.read_excel(file, sheet_name=sheet_name, skiprows=2)
#         print(sheet_name, "rows: ")
        # Ensure we have the correct number of columns
        if df.shape[1] == 4:
            df.columns = ["Photo #", "Weight (1-10)", "Gender", "Race"]
            df["Notes"] = ""
        elif df.shape[1] == 5:
            df.columns = ["Photo #", "Weight (1-10)", "Gender", "Race", "Notes"]
        else:
            continue
        
        df["Weight (1-10)"] = df["Weight (1-10)"].apply(to_tuple)
        df["Gender"] = df["Gender"].apply(to_tuple)
        df["Race"] = df["Race"].apply(to_tuple)
        
        df["RATER"] = rater
        df["LABEL"] = sheet_name

        dataframes.append(df)

final_df = pd.concat(dataframes, ignore_index=True)
final_df = final_df[['RATER', 'LABEL', 'Photo #', "Weight (1-10)", "Gender", "Race"]]
cleaned_df = final_df.copy()

cleaned_df['WeightRate_Person1'] = cleaned_df['Weight (1-10)'].map(lambda x: float(x[0]))
cleaned_df['WeightRate_Person2'] = cleaned_df['Weight (1-10)'].map(lambda x: float(x[1]) if len(x)>1 else pd.NA)
cleaned_df['Gender_Person1'] = cleaned_df['Gender'].map(lambda x: x[0])
cleaned_df['Gender_Person2'] = cleaned_df['Gender'].map(lambda x: x[1] if len(x)>1 else pd.NA)
cleaned_df['Race_Person1'] = cleaned_df['Race'].map(lambda x: x[0])
cleaned_df['Race_Person2'] = cleaned_df['Race'].map(lambda x: x[1] if len(x)>1 else pd.NA)

cleaned_df.drop(['Weight (1-10)', 'Gender', 'Race'], axis=1, inplace=True)

consolidated = cleaned_df.groupby(['LABEL','Photo #'])[["WeightRate_Person1",'WeightRate_Person2','Gender_Person1','Gender_Person2','Race_Person1','Race_Person2']].agg({
                    'WeightRate_Person1': pd.Series.mean,
                    'WeightRate_Person2': pd.Series.mean,
                    'Gender_Person1': pd.Series.mode,
                    'Gender_Person2': pd.Series.mode,
                    'Race_Person1': pd.Series.mode,
                    'Race_Person2': pd.Series.mode
                }).reset_index()

consolidated.to_excel(os.path.join(directory, 'consolidated.xlsx'), index=False)

print("Data combined successfully!")

Data combined successfully!


In [79]:
for rater in final_df.RATER.unique():
    print(rater, len(final_df[(final_df['RATER']==rater)].sort_values(by='LABEL').LABEL.unique()))

for label in final_df.LABEL.unique():
    c_labels = len(final_df[(final_df['LABEL']==label)].sort_values(by='RATER').RATER.unique())
    if c_labels != 4:
        print(label)

JANE 40
AKASH 40
DAVID 40
ANNIKA 40
